In [19]:
# Import Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [20]:
# Function to remove formatting from gatherer
def extract_name(card, card_long):
    length = len(card_long)
    good_string = card_long[length-len(card):-1]
    good_string += card_long[length-1]
    return(good_string) 

In [21]:
# Create browser to navigate oracle
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://gatherer.wizards.com/Pages/Default.aspx'
browser.visit(url)

In [22]:
# Here will be the list with all cards which will be converted from a csv.
# Temporarily, these cards are chosen so each mana symbol is represented at least once.
cards_df = pd.DataFrame({"Name": ["Standard Bearer", "Ponder", "Pestilence", "Lightning Bolt", "Forest"],
                         "Copies": [0,1,2,3,4]
                        })

In [23]:
# Cards dataframe is used to scrape oracle text and then relevant data is crafted into dataframe

# Empty list for list of mismatched links is crafted
bad_link = []

# Empty lists are initialized
card_list = []
copies_list = []
gen_list = []
snow_list = []
c_list = []
v_list = []
two_b_list = []
phy_list = []
w_list = []
u_list = []
b_list = []
r_list = []
g_list = []
cmc_list = []
type_line_list = []
text_list = []
power_list = []
toughness_list = []
loyalty_list = []

# For loop loops through list of cards
for row in cards_df.iterrows():
    
    # Initialize all mana values which may be duplicated at 0
    gen = 0
    snow = 0
    c = 0
    v = 0
    two_b = 0
    phy = 0
    w = 0
    u = 0
    b = 0
    r = 0
    g = 0
    
    # Navigates browser to card page.  Note that this picks the first option in the dropdown when a card name is entered.
    # It is possible that the wrong card will be grabbed, so this is verified below.
    browser.fill("ctl00$ctl00$MainContent$Content$SearchControls$CardSearchBoxParent$CardSearchBox", row[1]["Name"])
    time.sleep(1)
    browser.links.find_by_partial_text(row[1]["Name"]).click()
    
    # Soup object is crafted at card page so html can be read
    html = browser.html
    hot_soup = BeautifulSoup(html, "html.parser")
    
    # Check to see if card navigation was successful.  Name must be extracted from formatting to check strings are equal.
    # If strings are not equal, the entry is entered into a list.  Otherwise, the program continues.
    card_long = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_nameRow").text.split("\n")[1]
    card_name = extract_name(row[1]["Name"], card_long)
    if row[1]["Name"] != card_name:
        bad_link.append([row[1]["Name"],row[1]["Copies"]])
    
    # Card name is correct,so data is scraped
    else:
        
        # Check for a mana cost, if one does not exist, skip ahead
        try:
            mana_row = hot_soup.find("div", class_="row manaRow").find_all("img")
            
            # Loop through all mana symbols in the mana row, in each case the alt text will be used.
            # Mana is saved as [gen]eric, [snow], [c]olorless, [v]ariable, (two_b)rid, (phy)rexian, and
            #    [w]hite, bl[u]e, [b]lack, [r]ed, and [g]reen (hybrid, phyrexian, and 2brid mana are counted twice).
            for cost in mana_row:
                
                # Attempt to turn symbol into a number, only possible if it is generic
                try:
                    gen = int(cost["alt"])
                
                # The symbol is not generic mana
                except:
                    
                    # These three qualities are unique, and thus mutually exclusive
                    if "Snow" == cost["alt"]:
                        snow += 1
                    elif "Colorless" == cost["alt"]:
                        c += 1
                    elif "Variable Colorless" == cost["alt"]:
                        v += 1
                    
                    # The following mana types may appear in combination in the same symbol, thus are checked if the string is
                    #    in the alt text.  The intent is that mana which may be paid for in multiple ways is counted as such.
                    else:
                        if "Two or " in cost["alt"]:
                            two_b += 1
                        if "Phyrexian" in cost["alt"]:
                            phy += 1
                        if "White" in cost["alt"]:
                            w += 1
                        if "Blue" in cost["alt"]:
                            u += 1
                        if "Black" in cost["alt"]:
                            b += 1
                        if "Red" in cost["alt"]:
                            r += 1
                        if "Green" in cost["alt"]:
                            g += 1    

        
        # If no mana cost, return all mana entries as None ("None" is different from "0", see Mountain and Ornithopter)
        except:
            gen = None
            snow = None
            c = None
            v = None
            two_b = None
            phy = None
            w = None
            u = None
            b = None
            r = None
            g = None

        
        
        # Scrape a converted mana cost. If one does not exist, skip ahead
        # In order to ensure that the cmc is a number and usable, it must be altered from that on oracle.
        # Here, we abuse the fact that the highest cmc in pauper is 9, so all numbers will be 1 character.
        try:
            ugly_number = hot_soup.find_all("div", class_="row")[2].find("div", class_="value").text
            cmc = int(ugly_number[len(ugly_number) - 1])
        except:
            cmc = None
        
        # Scrape the typeline
        type_line = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_typeRow").text.split("\n")[1]
        
        # Check for a text box, if one does not exist, skip ahead
        try:
            text = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow").\
                text.replace("Card Text:\n", "")
        except:
            text = None
        
        # Check for power and toughness.  If both are valid, set loyalty to zero.
        try:
            pt = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow").text.split("\n")[1].\
                    split(" / ")
            power = int(pt[0])
            toughness = int(pt[1])
            loyalty = None
        # Something failed so card has no card has no pt. Next, check if card is a planeswalker.
        except:
            power = None
            toughness = None
            try:
                loyalty = int(browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow").text.\
                              split("\n")[1])
            except:
                loyalty = None
    
    # Appends the card data to the relevant lists
    card_list.append(row[1]["Name"])
    copies_list.append(row[1]["Copies"])
    gen_list.append(gen)
    snow_list.append(snow)
    c_list.append(c)
    v_list.append(v)
    two_b_list.append(two_b)
    phy_list.append(phy)
    w_list.append(w)
    u_list.append(u)
    b_list.append(b)
    r_list.append(r)
    g_list.append(g)
    cmc_list.append(cmc)
    type_line_list.append(type_line)
    text_list.append(text)
    power_list.append(power)
    toughness_list.append(toughness)
    loyalty_list.append(loyalty)
    
    # Navigates back to search page
    browser.back()

In [24]:
card_df= pd.DataFrame({"Name": card_list,
                       "Copies": copies_list,
                       "Cost Generic": gen_list,
                       "Cost Snow": snow_list,
                       "Cost Colorless": c_list,
                       "Cost Variable": v_list,
                       "Cost 2-brid": two_b_list,
                       "Cost Phyrexian": phy_list,
                       "Cost White": w_list,
                       "Cost Blue": u_list,
                       "Cost Black": b_list,
                       "Cost Red": r_list,
                       "Cost Green": g_list,
                       "Converted Mana Cost": cmc_list,
                       "Type Line": type_line_list,
                       "Text Box": text_list,
                       "Power": power_list,
                       "Toughness": toughness_list,
                       "Loyalty": loyalty_list
                      })
card_df

,Name,Copies,Cost Generic,Cost Snow,Cost Colorless,Cost Variable,Cost 2-brid,Cost Phyrexian,Cost White,Cost Blue,Cost Black,Cost Red,Cost Green,Converted Mana Cost,Type Line,Text Box,Power,Toughness,Loyalty
0,Standard Bearer,0,1,0,0,0,0,0,1,0,0,0,0,2,Creature — Human Flagbearer,While an opponent is choosing targets as part ...,1.0,1.0,None
1,Ponder,1,0,0,0,0,0,0,0,1,0,0,0,1,Sorcery,"Look at the top three cards of your library, t...",NaN,NaN,None
2,Pestilence,2,2,0,0,0,0,0,0,0,2,0,0,4,Enchantment,"At the beginning of the end step, if no creatu...",NaN,NaN,None
3,Lightning Bolt,3,0,0,0,0,0,0,0,0,0,1,0,1,Instant,Lightning Bolt deals 3 damage to any target.,NaN,NaN,None
4,Forest,4,0,0,0,0,0,0,0,0,0,0,0,1,Instant,Lightning Bolt deals 3 damage to any target.,NaN,NaN,None


In [25]:
# Generate a list of card names with bad links, and a list of copies of cards with bad links

# Empty lists initialized
bad_names = []
bad_copies = []

# For loop floops through the links appending relevant entries to relevant list
for link in bad_link:
    bad_names.append(link[0])
    bad_copies.append(link[1])
    
# Turns lists into a dataframe for later merging
bad_df = pd.DataFrame({"Name": bad_names,
                       "Copies": bad_copies})
bad_df

,Name,Copies
0,Forest,4


In [26]:
# Drop all cards whose link did not go to the correct card
card_df.drop(card_df.loc[card_df["Name"].isin(bad_names)].index, inplace=True)

In [27]:
# A dataframe of cards known to cause issues when the above process is run is manually entered then merged with the bad_df.
bad_ids = pd.DataFrame({"Name": ["Island", "Swamp", "Mountain", "Forest", "Ponder"],
                       "Oracle ID": [491574, 491576, 491578, 491580, 451051]})
card_id_df = bad_df.merge(bad_ids, how="inner", on="Name")
card_id_df

,Name,Copies,Oracle ID
0,Forest,4,491580


In [28]:
# Dataframe of cards with bad links has data scraped using card url's

# Base url for manual entry and navigation
base_url = "https://gatherer.wizards.com/Pages/Card/Details.aspx?multiverseid="

# Empty lists are initialized
card_list = []
copies_list = []
gen_list = []
snow_list = []
c_list = []
v_list = []
two_b_list = []
phy_list = []
w_list = []
u_list = []
b_list = []
r_list = []
g_list = []
cmc_list = []
type_line_list = []
text_list = []
power_list = []
toughness_list = []
loyalty_list = []

# For loop loops through list of cards
for row in card_id_df.iterrows():
    
    url = base_url + str(row[1]["Oracle ID"])
    
    # Initialize all mana values which may be duplicated at 0
    gen = 0
    snow = 0
    c = 0
    v = 0
    two_b = 0
    phy = 0
    w = 0
    u = 0
    b = 0
    r = 0
    g = 0
    
    # Navigates browser to card page.  Note that this picks the first option in the dropdown when a card name is entered.
    # It is possible that the wrong card will be grabbed, so this is verified below.
    browser.visit(url)
    time.sleep(1)
    
    # Soup object is crafted at card page so html can be read
    html = browser.html
    hot_soup = BeautifulSoup(html, "html.parser")
    
    
    # Check for a mana cost, if one does not exist, skip ahead
    try:
        mana_row = hot_soup.find("div", class_="row manaRow").find_all("img")

        # Loop through all mana symbols in the mana row, in each case the alt text will be used.
        # Mana is saved as [gen]eric, [snow], [c]olorless, [v]ariable, (two_b)rid, (phy)rexian, and
        #    [w]hite, bl[u]e, [b]lack, [r]ed, and [g]reen (hybrid, phyrexian, and 2brid mana are counted twice).
        for cost in mana_row:

            # Attempt to turn symbol into a number, only possible if it is generic
            try:
                gen = int(cost["alt"])

            # The symbol is not generic mana
            except:

                # These three qualities are unique, and thus mutually exclusive
                if "Snow" == cost["alt"]:
                    snow += 1
                elif "Colorless" == cost["alt"]:
                    c += 1
                elif "Variable Colorless" == cost["alt"]:
                    v += 1

                # The following mana types may appear in combination in the same symbol, thus are checked if the string is
                #    in the alt text.  The intent is that mana which may be paid for in multiple ways is counted as such.
                else:
                    if "Two or " in cost["alt"]:
                        two_b += 1
                    if "Phyrexian" in cost["alt"]:
                        phy += 1
                    if "White" in cost["alt"]:
                        w += 1
                    if "Blue" in cost["alt"]:
                        u += 1
                    if "Black" in cost["alt"]:
                        b += 1
                    if "Red" in cost["alt"]:
                        r += 1
                    if "Green" in cost["alt"]:
                        g += 1    


    # If no mana cost, return all mana entries as None ("None" is different from "0", see Mountain and Ornithopter)
    except:
        gen = None
        snow = None
        c = None
        v = None
        two_b = None
        phy = None
        w = None
        u = None
        b = None
        r = None
        g = None



    # Scrape a converted mana cost. If one does not exist, skip ahead
    # In order to ensure that the cmc is a number and usable, it must be altered from that on oracle.
    # Here, we abuse the fact that the highest cmc in pauper is 9, so all numbers will be 1 character.
    try:
        ugly_number = hot_soup.find_all("div", class_="row")[2].find("div", class_="value").text
        cmc = int(ugly_number[len(ugly_number) - 1])
    except:
        cmc = None

    # Scrape the typeline
    type_line = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_typeRow").text.split("\n")[1]

    # Check for a text box, if one does not exist, skip ahead
    try:
        text = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow").\
            text.replace("Card Text:\n", "")
    except:
        text = None

    # Check for power and toughness.  If both are valid, set loyalty to zero.
    try:
        pt = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow").text.split("\n")[1].\
                split(" / ")
        power = int(pt[0])
        toughness = int(pt[1])
        loyalty = None
    # Something failed so card has no card has no pt. Next, check if card is a planeswalker.
    except:
        power = None
        toughness = None
        try:
            loyalty = int(browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow").text.\
                          split("\n")[1])
        except:
            loyalty = None
    
    # Appends the card data to the relevant lists
    card_list.append(row[1]["Name"])
    copies_list.append(row[1]["Copies"])
    gen_list.append(gen)
    snow_list.append(snow)
    c_list.append(c)
    v_list.append(v)
    two_b_list.append(two_b)
    phy_list.append(phy)
    w_list.append(w)
    u_list.append(u)
    b_list.append(b)
    r_list.append(r)
    g_list.append(g)
    cmc_list.append(cmc)
    type_line_list.append(type_line)
    text_list.append(text)
    power_list.append(power)
    toughness_list.append(toughness)
    loyalty_list.append(loyalty)

In [32]:
# The dataframe containing all cards whose links did not properly navigate is generated
card_manual_df= pd.DataFrame({"Name": card_list,
                              "Copies": copies_list,
                              "Cost Generic": gen_list,
                              "Cost Snow": snow_list,
                              "Cost Colorless": c_list,
                              "Cost Variable": v_list,
                              "Cost 2-brid": two_b_list,
                              "Cost Phyrexian": phy_list,
                              "Cost White": w_list,
                              "Cost Blue": u_list,
                              "Cost Black": b_list,
                              "Cost Red": r_list,
                              "Cost Green": g_list,
                              "Converted Mana Cost": cmc_list,
                              "Type Line": type_line_list,
                              "Text Box": text_list,
                              "Power": power_list,
                              "Toughness": toughness_list,
                              "Loyalty": loyalty_list
                      })
card_manual_df

,Name,Copies,Cost Generic,Cost Snow,Cost Colorless,Cost Variable,Cost 2-brid,Cost Phyrexian,Cost White,Cost Blue,Cost Black,Cost Red,Cost Green,Converted Mana Cost,Type Line,Text Box,Power,Toughness,Loyalty
0,Forest,4,None,None,None,None,None,None,None,None,None,None,None,None,Basic Land — Forest,G,None,None,None


In [34]:
# Dataframes of cards which did and did not navigate are appended to form one another to generate a complete dataframe
card_final_df = card_df.append(card_manual_df).reset_index(drop=True)
card_final_df.to_csv("mtg_merged_df.csv")
card_final_df

,Name,Copies,Cost Generic,Cost Snow,Cost Colorless,Cost Variable,Cost 2-brid,Cost Phyrexian,Cost White,Cost Blue,Cost Black,Cost Red,Cost Green,Converted Mana Cost,Type Line,Text Box,Power,Toughness,Loyalty
0,Standard Bearer,0,1,0,0,0,0,0,1,0,0,0,0,2,Creature — Human Flagbearer,While an opponent is choosing targets as part ...,1,1,None
1,Ponder,1,0,0,0,0,0,0,0,1,0,0,0,1,Sorcery,"Look at the top three cards of your library, t...",NaN,NaN,None
2,Pestilence,2,2,0,0,0,0,0,0,0,2,0,0,4,Enchantment,"At the beginning of the end step, if no creatu...",NaN,NaN,None
3,Lightning Bolt,3,0,0,0,0,0,0,0,0,0,1,0,1,Instant,Lightning Bolt deals 3 damage to any target.,NaN,NaN,None
4,Forest,4,None,None,None,None,None,None,None,None,None,None,None,None,Basic Land — Forest,G,None,None,None
